# NLP Tickets

Fine-Tuning модели для конкретной задачи.
</br>Взять предобученную модель (например, BERT, GPT или T5) и дообучить её на небольшом наборе данных для специализированной задачи:
</br>Классификация пользовательских запросов (например, техподдержка: инцидент, запрос и т.д.).

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
# Определим устройство: если доступен GPU, используем его, иначе — CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cuda')

In [4]:
# Загрузка данных из Excel файла
data_df = pd.read_excel("dataset-tickets_en.xlsx")
data_df

,body,type
0,I am reaching out regarding a high-priority ti...,Request
1,I am experiencing a high-priority incident whe...,Incident
2,I am writing to express our concern regarding ...,Change
3,I hope this message finds you well. I am writi...,Incident
4,I hope this message finds you well. My name is...,Request
...,...,...
3995,I hope this message finds you well. I am writi...,Incident
3996,I am contacting you to report a critical servi...,Request
3997,I am writing to bring to your attention a bill...,Incident
3998,I am facing a flickering issue on my Dell XPS ...,Incident


In [5]:
# Разделение данных на тренировочную и тестовую выборки
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data_df["body"], data_df["type"], test_size=0.2, random_state=42
)

In [6]:
train_texts

3994       My MacBook Air M1 is shutting down frequently.
423     \n\nI am experiencing issues with the ticket c...
2991     I am writing to express my concern regarding ...
1221    I am writing to inquire about certain charges ...
506     \n\nI hope this message finds you well. I'm re...
                              ...                        
1130    Having trouble accessing channels and dispatch...
1294    \n\nI am experiencing an issue with my Epson E...
860     \n\nI am writing to request the implementation...
3507    Dear Online Store Support Customer,<br><br>I h...
3174    Our customer, <name>, is seeking assistance in...
Name: body, Length: 3200, dtype: object

In [7]:
# Удаление строк с пустыми значениями в данных
data_df = data_df.dropna(subset=["body", "type"]).copy()

# Преобразование колонок в строковый тип (если значения в этих колонках могут быть числами)
data_df["body"] = data_df["body"].astype(str)
data_df["type"] = data_df["type"].astype(str)

# Разделение данных на тренировочную и тестовую выборки
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data_df["body"], data_df["type"], test_size=0.2, random_state=42
)

# Преобразование данных в формат Hugging Face Dataset
train_data = Dataset.from_dict({"text": train_texts.tolist(), "label": train_labels.tolist()})
test_data = Dataset.from_dict({"text": test_texts.tolist(), "label": test_labels.tolist()})

In [8]:
# Преобразование меток в числовой формат
label2id = {label: idx for idx, label in enumerate(data_df["type"].unique())}
id2label = {idx: label for label, idx in label2id.items()}
train_data = train_data.map(lambda x: {"label": label2id[x["label"]]})
test_data = test_data.map(lambda x: {"label": label2id[x["label"]]})

Map:   0%|          | 0/3199 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

## DistilBERT

Используем предобученную версию модели-трансформера DistilBERT, предоставленная библиотекой Hugging Face.
</br>DistilBERT основан на архитектуре трансформеров, как и его оригинальная модель — BERT (Bidirectional Encoder Representations from Transformers).
</br>DistilBERT: Это облегчённая версия модели BERT, оптимизированная для скорости и уменьшения объёма памяти без значительных потерь в точности.
</br>Uncased: Модель не учитывает регистр текста (всё преобразуется в нижний регистр).

</br>DistilBERT уже предобучен на большом корпусе текстов (например, из Википедии).
</br>Верхняя часть модели (голова) заменяется линейным слоем, который предназначен для классификации на основе выходов модели.

Что делает токенизатор?
* Преобразует текст в числовое представление, чтобы модель могла его понять.
* Разбивает текст на токены (слова или части слов) с использованием алгоритма BERT (WordPiece).
* Добавляет специальные токены, такие как [CLS] (для классификации) и [SEP] (разделитель).
* Проверяет, что длина текста соответствует ограничениям модели.

In [9]:
# Загрузка токенизатора и модели
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(label2id) # количество классов, которые модель должна предсказывать
)

# Переносим модель на нужное устройство
model.to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


Загрузка токенизатора (tokenizer = AutoTokenizer.from_pretrained(model_name)) предоставляет объект, который знает, как токенизировать текст. Однако сама токенизация текста происходит позже, при подготовке данных.
</br>Модель DistilBERT принимает на вход только числовые тензоры (Input IDs, Attention Masks и т.д.), а не текстовые строки. Этот процесс выполняется на этапе train_data.map(preprocess_data, batched=True).

In [10]:
# Токенизация данных
def preprocess_data(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=512)

train_data = train_data.map(preprocess_data, batched=True)
test_data = test_data.map(preprocess_data, batched=True)

Map:   0%|          | 0/3199 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [11]:
train_data

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 3199
})

In [12]:
test_data

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 800
})

In [13]:
# Удаление ненужных колонок
train_data = train_data.remove_columns(["text"])
test_data = test_data.remove_columns(["text"])

In [14]:
train_data

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 3199
})

DataCollatorWithPadding — это инструмент из библиотеки Hugging Face, который автоматически добавляет padding (дополнительные нули) к токенизированным последовательностям в батче так, чтобы все последовательности в батче имели одинаковую длину.
</br>Если один текст в батче состоит из 50 токенов, а другой — из 100, padding добавляет нули к более короткому тексту, чтобы его длина соответствовала самому длинному в батче.
</br>DataCollatorWithPadding автоматически подбирает максимальную длину токенов для каждого конкретного батча.

In [15]:
# Создание data collator для динамического паддинга
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [16]:
data_collator

DataCollatorWithPadding(tokenizer=DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
), padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

In [17]:
# Определение метрик для оценки
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

In [18]:
# Настройка параметров обучения
training_args = TrainingArguments(
    output_dir="./results", # Указывает путь к директории, где будут сохраняться результаты обучения.
    evaluation_strategy="epoch", # Определяет, как часто выполнять оценку модели (на тестовой выборке). "epoch": Оценка выполняется в конце каждой эпохи.
    save_strategy="epoch", # Определяет, как часто сохранять модель (чекпоинты). "epoch": Сохранение модели в конце каждой эпохи.
    learning_rate=2e-5, # Указывает скорость обучения (learning rate). Значение 2e-5: Это небольшая скорость обучения (0.00002), часто используемая для тонкой настройки моделей трансформеров.
    per_device_train_batch_size=4, # Устанавливает размер батча для тренировочных данных (на каждом устройстве, например, GPU). Значение 4: Это небольшой размер батча, подходящий для трансформеров, чтобы избежать переполнения памяти GPU. 
    per_device_eval_batch_size=4, # Устанавливает размер батча для тестовых данных (при оценке). Значение 4: Такое же, как для тренировочных данных, чтобы использовать ресурсы эффективно.
    num_train_epochs=10, #  Указывает количество эпох (полных проходов по всему тренировочному датасету). 
    weight_decay=0.01, # Добавляет L2-регуляризацию к весам модели, чтобы предотвратить переобучение. Значение 0.01: Рекомендуемое значение для трансформеров, чтобы слегка штрафовать слишком большие веса.
    logging_dir="./logs", # Указывает директорию, где будут сохраняться логи обучения.
    logging_steps=10, # Указывает, как часто (в шагах) логировать метрики. Значение 10: Каждые 10 шагов выводятся логи (например, потери и метрики).
    load_best_model_at_end=True, # Указывает, что в конце обучения нужно загрузить лучшую модель (на основе указанной метрики). Почему полезно: Если модель показывает наилучшие результаты не в последней эпохе, а в одной из предыдущих, она будет загружена.
    metric_for_best_model="accuracy", # Указывает, какая метрика используется для выбора лучшей модели. 
)

C:\Users\AADementev\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Создадим объект Trainer из библиотеки Hugging Face transformers, который автоматизирует процесс обучения, оценки и логирования модели.

In [19]:
# Создание объекта Trainer
trainer = Trainer(
    model=model, # Модель для обучения. Передаётся модель distilbert-base-uncased.
    args=training_args, # Перадаётся объект TrainingArguments, содержащий параметры обучения.
    train_dataset=train_data, # Датасет, используемый для обучения модели.
    eval_dataset=test_data, # Датасет для оценки (валидации) модели.
    tokenizer=tokenizer, # Токенизатор, соответствующий используемой модели.
    data_collator=data_collator, # Передаётся объект DataCollatorWithPadding. Упрощает обработку данных с разной длиной, динамически добавляя паддинг в батчах.
    compute_metrics=compute_metrics, # Передаётся пользовательская функция для вычисления метрик.
)

C:\Users\AADementev\AppData\Local\Temp\ipykernel_5952\1383386930.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [20]:
# Fine-Tuning модели
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.378500,1.258278,0.561250,0.429592,0.561250,0.466784
2,1.156200,1.096333,0.598750,0.557232,0.598750,0.560160
3,0.818900,1.072285,0.653750,0.637928,0.653750,0.628986
4,0.238300,1.108031,0.673750,0.664599,0.673750,0.667980
5,0.565200,1.318631,0.695000,0.684902,0.695000,0.686031
6,0.462700,1.362137,0.701250,0.711926,0.701250,0.704628
7,0.029300,1.455000,0.721250,0.709295,0.721250,0.713740
8,0.029400,1.558170,0.732500,0.719023,0.732500,0.723463
9,0.006000,1.612851,0.720000,0.715469,0.720000,0.717332
10,0.009600,1.640115,0.731250,0.724694,0.731250,0.727272


TrainOutput(global_step=8000, training_loss=0.5371529834524262, metrics={'train_runtime': 1323.7914, 'train_samples_per_second': 24.165, 'train_steps_per_second': 6.043, 'total_flos': 3782451716146164.0, 'train_loss': 0.5371529834524262, 'epoch': 10.0})

In [21]:
# Оценка модели
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 1.5581698417663574, 'eval_accuracy': 0.7325, 'eval_precision': 0.7190228353958595, 'eval_recall': 0.7325, 'eval_f1': 0.7234626713409501, 'eval_runtime': 9.0889, 'eval_samples_per_second': 88.02, 'eval_steps_per_second': 22.005, 'epoch': 10.0}


In [22]:
# Пример использования модели на новых данных
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    
    # Переносим входные данные на нужное устройство
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=-1).item()
    return id2label[prediction]

In [23]:
new_text = "I have a problem. My laptop is running slowly. Can you help me?"
print(f"Classification: {predict(new_text)}")

Classification: Request


In [24]:
new_text = "A system update is required, the settings have been lost."
print(f"Classification: {predict(new_text)}")

Classification: Incident
